In [1]:
%cd ../..
import math
import numpy as np
import time
from archive.python.LangevinGillespie import LangevinGillespie as LangevinGillespie_Legacy
from src.utils.compute_transition_matrix import compute_transition_matrix
from bin.f1sim import LangevinGillespie as LangevinGillespie_PybindWrap

/home/Robert/Code/Python/F1-ATPase-simulation


In [2]:
!fastfetch --logo none --structure os:kernel:memory:cpu:gpu

OS: Debian GNU/Linux 13 (trixie) x86_64
Kernel: Linux 6.6.87.2-microsoft-standard-WSL2
Memory: 3.73 GiB / 123.43 GiB (3%)
CPU: AMD Ryzen 9 9950X (32) @ 4.30 GHz
GPU 1: NVIDIA GeForce RTX 5080 (15.60 GiB) [Discrete]
GPU 2: AMD Radeon(TM) Graphics (1.97 GiB) [Integrated]


In [3]:
def initialize_simulation_params(LG):
    LG.steps = 100_000
    LG.dt = 1e-6
    LG.method = "heun"

    # Mechanical / Thermal Setup
    LG.kappa = 56
    LG.kBT = 4.14
    LG.gammaB = LG.computeGammaB(a=20, r=19, eta=1e-9)

    # Multi State Setup
    LG.theta_states = np.array([3, 36, 72, 116]) * math.pi / 180  # Deg → Rad
    LG.initial_state = 0  # Starting state

    # Transition rate matrix
    LG.transition_matrix = compute_transition_matrix(LG)

In [4]:
# Initialize simulation wrapper
LG_PybindWrap = LangevinGillespie_PybindWrap()
LG_Legacy = LangevinGillespie_Legacy()
initialize_simulation_params(LG_PybindWrap)
initialize_simulation_params(LG_Legacy)

N_SIMS = 10_000
times = {}

##### Note: If you use multi-threading, avoid using swap memory, instead employ batching

In [5]:
print(f"\nRunning {N_SIMS} simulations on the GPU...")
start_time = time.time()

combined_bead_positions_gpu, combined_states_gpu, combined_target_thetas_gpu = (
    LG_PybindWrap.simulate_multithreaded_cuda(N_SIMS, seed=101)
)

# Store the result in the dictionary
times["CUDA"] = time.time() - start_time
print(f"  GPU simulation complete in: {times['CUDA']:.4f} seconds")
combined_bead_positions_gpu, combined_states_gpu, combined_target_thetas_gpu = (
    None,
    None,
    None,
)


Running 10000 simulations on the GPU...
  GPU simulation complete in: 15.1812 seconds


In [6]:
CPU_THREAD_NUMBER = 32

print(f"Running {N_SIMS} simulations on {CPU_THREAD_NUMBER} CPU threads...")
start_time = time.time()

# Call the function ONCE with the total number of sims
combined_bead_positions_cpu, combined_states_cpu, combined_target_thetas_cpu = (
    LG_PybindWrap.simulate_multithreaded(
        N_SIMS, 
        CPU_THREAD_NUMBER, 
        seed=101
    )
)

# Store the result in the dictionary
times["Multithreaded"] = time.time() - start_time
print(f"  CPU simulation complete in: {times['Multithreaded']:.4f} seconds")
combined_bead_positions_cpu, combined_states_cpu, combined_target_thetas_cpu = None, None, None

# --- Final Comparison ---
print("\n" + "="*30)
print(f"Speedup (GPU vs CPU): {times['Multithreaded'] / times['CUDA']:.2f}x")
print("="*30)

Running 10000 simulations on 32 CPU threads...
  CPU simulation complete in: 24.6841 seconds

Speedup (GPU vs CPU): 1.63x


In [7]:
bead_store = []
states_store = []
thetas_store = []
start_time = time.time()

for i in range(N_SIMS):
    beads, states, thetas = LG_PybindWrap.simulate()
    bead_store.append(beads)
    states_store.append(states)
    thetas_store.append(thetas)

times["SingleThreaded"] = time.time() - start_time
print(f"\nTotal time: {times['SingleThreaded']:.2f} s")
bead_store, states_store, thetas_store = None, None, None


Total time: 104.37 s


In [8]:
bead_store = []
states_store = []
thetas_store = []
start_time = time.time()

for i in range(N_SIMS):
    beads, states, thetas = LG_Legacy.simulate()
    bead_store.append(beads)
    states_store.append(states)
    thetas_store.append(thetas)

times["Legacy"] = time.time() - start_time
print(f"\nTotal time: {times['Legacy']:.2f} s")
bead_store, states_store, thetas_store = None, None, None


Total time: 5987.23 s


In [9]:
# ============================================================
#                      COLOR DEFINITIONS
# ============================================================


def rgb_bg(r, g, b):
    return f"\033[48;2;{r};{g};{b}m"


def rgb_fg(r, g, b):
    return f"\033[38;2;{r};{g};{b}m"


RESET = "\033[0m"

# Clean background colors
COLOR_GREEN = rgb_bg(130, 220, 130)
COLOR_BLUE = rgb_bg(130, 160, 230)
COLOR_RED = rgb_bg(230, 140, 140)
COLOR_MAGENTA = rgb_bg(200, 150, 230)

# Dark readable foreground text
FG_DARK = rgb_fg(20, 20, 20)  # nearly black

ENGINE_COLORS = {
    "cuda": COLOR_GREEN,
    "multithreaded": COLOR_BLUE,
    "singlethreaded": COLOR_MAGENTA,
    "legacy": COLOR_RED,
}


def color_engine(name, padded):
    """Wrap padded text in background + dark text if it matches any engine."""
    key = name.lower()
    if key in ENGINE_COLORS:
        return f"{ENGINE_COLORS[key]}{FG_DARK}{padded}{RESET}"
    return padded


# ============================================================
#                      ORIGINAL TABLE CODE
# ============================================================

spacing = 17

chart_information = (
    "║" + f"{'Name':^{spacing}}║"
    f"{'Time':^{spacing}}║"
    f"{'Comparison':^{spacing}}║"
    f"{'Comparison Time':^{spacing}}║"
    f"{'Speed Up':^{spacing}}║"
)

line_char = "═"
title = f"Time comparisons for {N_SIMS} simulations and {LG_PybindWrap.steps} steps"
top_line_str = (line_char * spacing + "╦") * 5
bot_line_str = (line_char * spacing + "╩") * 5
bar_line_str = "║" + (line_char * spacing + "╬") * 5

# Top border + title
print("╔" + top_line_str.replace("╦", "═")[: len(top_line_str) - 1] + "╗")
print("║" + f"{title:^{(len(top_line_str) - 1)}}" + "║")
print("║" + top_line_str[: len(top_line_str) - 1] + "║")

# Header row
print(chart_information)

# Data rows
for key, value in times.items():
    for key_comparison, value_comparison in times.items():
        if key != key_comparison and value < value_comparison:
            speed_up = (value_comparison / value)

            print(bar_line_str[: len(bar_line_str) - 1] + "║")

            name_cell = color_engine(key, f"{key:^{spacing}}")
            comp_cell = color_engine(key_comparison, f"{key_comparison:^{spacing}}")

            print(
                f"║{name_cell}"
                f"║{f'{value:.2f}s':^{spacing}}"
                f"║{comp_cell}"
                f"║{f'{value_comparison:.2f}s':^{spacing}}"
                f"║{f'{speed_up:.2f}':^{spacing}}║"
            )

# Bottom border
print("╚" + bot_line_str[: len(bot_line_str) - 1] + "╝")


╔═════════════════════════════════════════════════════════════════════════════════════════╗
║                 Time comparisons for 10000 simulations and 100000 steps                 ║
║═════════════════╦═════════════════╦═════════════════╦═════════════════╦═════════════════║
║      Name       ║      Time       ║   Comparison    ║ Comparison Time ║    Speed Up     ║
║═════════════════╬═════════════════╬═════════════════╬═════════════════╬═════════════════║
║      CUDA       ║     15.18s      ║  Multithreaded  ║     24.68s      ║      1.63       ║
║═════════════════╬═════════════════╬═════════════════╬═════════════════╬═════════════════║
║      CUDA       ║     15.18s      ║ SingleThreaded  ║     104.37s     ║      6.87       ║
║═════════════════╬═════════════════╬═════════════════╬═════════════════╬═════════════════║
║      CUDA       ║     15.18s      ║     Legacy      ║    5987.23s     ║     394.39      ║
║═════════════════╬═════════════════╬═════════════════╬═════════════════╬═══════